In [ ]:
!python -V

Python 3.10.12


In [1]:
import pandas as pd

In [2]:
import pickle

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_squared_error

In [5]:
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
!wget https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet

--2024-05-18 12:20:31--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.237.12, 13.226.237.29, 13.226.237.221, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.237.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 47673370 (45M) [application/x-www-form-urlencoded]
Saving to: ‘yellow_tripdata_2023-01.parquet’

yellow_tripdata_202 100%[===================>]  45.46M  68.2MB/s    in 0.7s    

2024-05-18 12:20:32 (68.2 MB/s) - ‘yellow_tripdata_2023-01.parquet’ saved [47673370/47673370]

--2024-05-18 12:20:32--  https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.226.237.12, 13.226.237.29, 13.226.237.221, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.226.237.12|:443... conne

In [12]:
df_yellow_jan = pd.read_parquet('yellow_tripdata_2023-01.parquet')
print("No of columns in Jan Data: ", len(df_yellow_jan.columns))
# df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
# df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

# df = df[(df.duration >= 1) & (df.duration <= 60)]

# categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

# df[categorical] = df[categorical].astype(str)

No of columns in Jan Data:  19


In [13]:
df_yellow_jan.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


Q2. Computing duration
Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [ ]:
df_yellow_jan['duration'] = df_yellow_jan.tpep_dropoff_datetime - df_yellow_jan.tpep_pickup_datetime
df_yellow_jan.duration = df_yellow_jan.duration.apply(lambda td: td.total_seconds()/60)


In [22]:
duration_std = df_yellow_jan.duration.std()
print("The standard deviation of the trips duration in January is :", duration_std)

The standard deviation of the trips duration in January is : 42.59435124195458


In [35]:
def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

Q3. Dropping outliers
Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

In [24]:
actual_row_count = df_yellow_jan.shape[0]
df_yellow_jan_cleaned = df_yellow_jan[(df_yellow_jan.duration>=1) & (df_yellow_jan.duration<=60)]
row_count_post_outliers = df_yellow_jan_cleaned.shape[0]
print("Fraction of the records left after dropping the outliers : ", (row_count_post_outliers/actual_row_count)*100)

Fraction of the records left after dropping the outliers :  98.1220282212598


Q4. One-hot encoding
Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
Fit a dictionary vectorizer
Get a feature matrix from it

In [27]:
categorical = ['PULocationID', 'DOLocationID']
# numerical = ['trip_distance']

df_yellow_jan_cleaned[categorical] = df_yellow_jan_cleaned[categorical].astype(str)
train_dicts = df_yellow_jan_cleaned[categorical].to_dict(orient='records')
train_dicts[0:3]

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



<ipython-input-27-2ee86981a827>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_yellow_jan_cleaned[categorical] = df_yellow_jan_cleaned[categorical].astype(str)


The dimensionality of the feature matrix (number of columns) is:  515


In [29]:
print("The dimensionality of the feature matrix (number of columns) is: ",X_train.shape[1])

The dimensionality of the feature matrix (number of columns) is:  515


Q5. Training a model
Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters
Calculate the RMSE of the model on the training data

In [37]:
df_train = read_dataframe("yellow_tripdata_2023-01.parquet")
df_val = read_dataframe("yellow_tripdata_2023-02.parquet")

In [38]:
categorical = ['PULocationID', 'DOLocationID']

dv = DictVectorizer()

train_dicts = df_train[categorical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [39]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [41]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_val)
y_pred_train = lr.predict(X_train)


In [42]:
print("RMSE on Validation: ",mean_squared_error(y_val, y_pred, squared=False))
print( "RMSE on train: ",mean_squared_error(y_train, y_pred_train, squared=False))


RMSE on Validation:  7.811818933419717
RMSE on train:  7.649261929771859
